<a href="https://colab.research.google.com/github/iesous-kurios/family_promise_data/blob/master/Last_Updated_YTD_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **YTD FPS Data Notebook**

In [259]:
#@title
# import libraries which provide math functions to our data imports
import statistics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xlwt 
from xlwt import Workbook
import seaborn as sns

import matplotlib.pyplot as plt
import matplotlib
from matplotlib import style

sns.__version__

'0.9.1'

In [0]:
#@title
# import weekly data from yearly FPS folder
df = pd.read_excel('/content/90Day_Feb5.xlsx')
#df = pd.read_excel ('~/environments/fp_env/family_promise_data/data/ClientTrackExport (5).xlsx')

In [0]:
# get list of currently enrolled guests (guests that don't have an exit date)
currently_enrolled = df[df['3.11 Exit Date'].isnull()]

In [0]:
# create new list of those currently enrolled who haven't stayed with us for
# two weeks or more
need_to_exit = currently_enrolled[currently_enrolled['Date of Last ES Stay (Beta)'] < pd.to_datetime('2020-01-10')] 

In [0]:
# make list of last names and HMIS number so team can find them and exit them

last_names = need_to_exit['3.1 LastName']
hmis = need_to_exit['5.8 Personal ID']

print_list = pd.concat([last_names, hmis], axis=1)

In [0]:
# send to excel sheet to download and share
print_list.to_excel('/content/exit_list.xlsx', index=None)

In [0]:
# Python program to check if two  
# to get unique values from list 
# using traversal  
  
# function to get unique values 
def unique(list1): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x) 
    return unique_list    
    
  
# driver code 
list1 = df['5.8 Personal ID']

unduplicated = unique(list1) 

In [266]:
len(unduplicated)

266

In [267]:
df['5.9 Household ID'].value_counts()

126179    8
126161    6
120744    6
125108    6
121337    5
         ..
123342    2
124766    2
125107    2
126178    1
125339    1
Name: 5.9 Household ID, Length: 89, dtype: int64

In [0]:
# print total number of families served (unduplicated)
unique_households = unique(df['5.9 Household ID'])

In [269]:
len(unique_households)

89

In [0]:
#@title
# replace zeros in data with "Nan" to calculate more accurate average bednights 
df['Count of Bed Nights (Housing Check-ins)'] = df['Count of Bed Nights (Housing Check-ins)'].replace(0, np.NaN)

In [271]:
# display all possible exit destinations and their totals
df['3.12 Exit Destination'].value_counts()

Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter    34
No exit interview completed                                                                                             23
Client refused                                                                                                          16
Rental by client, no ongoing housing subsidy                                                                            15
Rental by client with RRH or equivalent subsidy                                                                         10
Rental by client, other ongoing housing subsidy                                                                          4
Other                                                                                                                    4
Substance Abuse Treatment or Detox Center                                                                                2
Name: 3.12 Exit 

In [0]:
#@title

exit_reasons = ['Rental by client with RRH or equivalent subsidy', 
                'Rental by client, no ongoing housing subsidy', 
                'Staying or living with family, permanent tenure', 
                'Rental by client, other ongoing housing subsidy',
                'Permanent housing (other than RRH) for formerly homeless persons', 
                'Staying or living with friends, permanent tenure', 
                'Owned by client, with ongoing housing subsidy', 
                'Staying or living with friends, permanent tenure', 
                'Rental by client, VASH housing Subsidy'
               ]

temp_reasons = ['Staying or living with family, temporary tenure (e.g., room, apartment or house)', 
               'Staying or living with friends, temporary tenure (e.g., room, apartment or house)', 
                
               ]

In [0]:
# drop rows with no exit destination (current guests at time of report)
df = df.dropna(subset=['3.12 Exit Destination'])

In [0]:
 # create target column (multiple types of exits to perm)
df['perm_leaver'] = df['3.12 Exit Destination'].isin(exit_reasons)

In [275]:
df['perm_leaver'].value_counts(normalize=True)

False    0.731481
True     0.268519
Name: perm_leaver, dtype: float64

In [276]:
#@title  { display-mode: "both" }
# pull all exit destinations from main data file and sum up the totals of each destination, placing them into new df for calculations
exits = df['3.12 Exit Destination'].value_counts()

tempExits = df['3.12 Exit Destination'].isin(temp_reasons)
temp_exit_percent = tempExits.sum() / exits.sum()
#exits.sum()) * 100)
# round tempExits to two decimals
tempExits = round(temp_exit_percent*100, 2)
#tempExits = 0

# take sum of unknown exits and divide by total exits, then multiply by 100 to get percentage
unknownExits = ((exits['Client refused'] / exits.sum()) * 100)

# round the answer to nearest hundreth place
unknownExits = (round(unknownExits, 2))

# create three data frames made up of the three key words found in perm housing
perm1 = df[df['3.12 Exit Destination'].str.contains('Rental') == True] 
perm2 = df[df['3.12 Exit Destination'].str.contains('permanent') == True]
perm3 = df[df['3.12 Exit Destination'].str.contains('Permanent') == True]

# merge the three dataframes into one to calculate exit to perm percent
perm_almost_final = perm1.append(perm2)
perm_final = perm_almost_final.append(perm3)
perm_final['3.12 Exit Destination'].value_counts()
perm_final_sums = perm_final['3.12 Exit Destination'].value_counts()
perm_final = df['3.12 Exit Destination'].isin(exit_reasons)
# calculate percentage of exits to perm
perm_final_percent = perm_final_sums.sum() / exits.sum()

#perm_final.sum()

#perm_final_percent2 = perm_final.sum() / df['3.12 Exit Destination'].value_counts().sum()
#round(perm_final_percent1, 2)
print('Exit to Perm percent:',perm_final_percent*100,'%')
print('Unknown Exit percent:',unknownExits,'%')
print('Temp Exit percent:',tempExits,'%')

Exit to Perm percent: 26.851851851851855 %
Unknown Exit percent: 14.81 %
Temp Exit percent: 0.0 %


In [277]:
#@title
# take sum of ES and Trans exits and divide by total exits, then multiply by 100 to get percentage
esExits = (( exits['Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter'] / exits.sum()) * 100)
#transExits = ((exits['Transitional Housing for homeless persons (including homeless youth)'] / exits.sum()) * 100)
# round the answer to nearest hundreth place
esExits = (round(esExits, 2))
#transExits = (round(transExits, 2))

print('Exits to ES percent:',esExits,'%')
print('Exits to Transitional',transExits,'%')

Exits to ES percent: 31.48 %
Exits to Transitional 5.71 %


In [278]:
#@title

avgBedNights = df['Count of Bed Nights (Housing Check-ins)'].mean(skipna=True)

avgBedNights = round(avgBedNights, 2)

print("Average BedNights is:", avgBedNights)

Average BedNights is: 29.37


In [279]:
#@title
avgDaysEnrolled = df['Days Enrolled in Project'].mean(skipna=True)
avgDaysEnrolled = round(avgDaysEnrolled, 2)                   
perm_final_percent = round(perm_final_percent, 2)
print('Average Days Enrolled is:', avgDaysEnrolled)

Average Days Enrolled is: 38.17


In [280]:
#@title
print('Average Household Income at Enrollment is:', round(df['4.2 Income Total at Entry'].mean(skipna=True), 2))

print('Average Household Income at Exit is:', round(df['4.2 Income Total at Exit'].mean(skipna=True), 2))

avgChange = (df['4.2 Income Total at Exit'].mean(skipna=True)) - (df['4.2 Income Total at Entry'].mean(skipna=True))
avgChange = round(avgChange, 2)
print('Average Change in Income for Households during stay is:', avgChange)


Average Household Income at Enrollment is: 887.17
Average Household Income at Exit is: 1061.87
Average Change in Income for Households during stay is: 174.69


In [281]:
#@title
# printed rounded answer for average age during enrollment
print('Average Age of Guest during Enrollment is:', round(df['Age at Enrollment'].mean(skipna=True), 2))

Average Age of Guest during Enrollment is: 20.29


In [0]:
# export zips into an excel sheet for Power BI visual
#  df['V5 Zip'].value_counts().to_excel('/content/drive/My Drive/FPS Weekly Data Colabs/YTD Data/weeklyYTDzips.xlsx', header=True, index=True)

In [0]:
# create list of all options for engagement data and export into excel file for Power BI
#df['Days Enrolled Until Engagement Date'].value_counts().to_excel('/content/drive/My Drive/FPS Weekly Data Colabs/YTD Data/weeklyYTD_aggregate_DOE.xlsx', header=True, index=True)

In [0]:
# create new excel sheet with the days enrolled until engagement to upload back into notebook for calculating mean

df['Days Until Engagement'] = (df['4.13 Engagement Date'] - df['3.10 Enroll Date'])
df['Days Until Engagement'].mean()
d_o_e_avg = df['Days Until Engagement'].mean()

In [285]:
d_o_e_avg

Timedelta('4 days 11:22:06.315789')

In [0]:
# create new excel sheet with the days enrolled until engagement to upload back into notebook for calculating mean 
#(df['4.13 Engagement Date'] - df['3.10 Enroll Date']).to_excel('/content/drive/My Drive/FPS Weekly Data Colabs/YTD Data/ytdDOE.xlsx', header=False, index=False)

In [0]:
# create new column with total amount of guests for year
df['Total Guests'] = df['5.8 Personal ID'].drop_duplicates().count()

In [0]:
# create new column with count of unique households
df['Total Households'] = df['5.9 Household ID'].drop_duplicates().count()

# create new column with total amount of guests for year
df['Total Guests'] = df['5.8 Personal ID'].drop_duplicates().count()


In [289]:
df['5.8 Personal ID'].count()

108

In [290]:
df['5.8 Personal ID'].drop_duplicates().count()

108

In [291]:
df['Age at Enrollment'].head()

0     29
1     28
2      3
23    54
24    17
Name: Age at Enrollment, dtype: int64

In [0]:
adults = df[df['Age at Enrollment'] > 17.0]

In [293]:
adults['4.05 Physical Disability'].value_counts()

Physical Disability    8
Name: 4.05 Physical Disability, dtype: int64

In [294]:
adults['4.07 Chronic Health Condition'].value_counts()

Chronic Health    13
Name: 4.07 Chronic Health Condition, dtype: int64

In [295]:
adults['4.06 Developmental Disability'].value_counts()

Developmental Disability    10
Name: 4.06 Developmental Disability, dtype: int64

In [296]:
65+77+37

179

In [0]:
minors = df[df['Age at Enrollment'] <= 17.0]

In [298]:
minors['4.05 Physical Disability'].value_counts()

Series([], Name: 4.05 Physical Disability, dtype: int64)

In [299]:
minors['4.07 Chronic Health Condition'].value_counts()

Chronic Health    4
Name: 4.07 Chronic Health Condition, dtype: int64

In [300]:
minors['4.06 Developmental Disability'].value_counts()

Developmental Disability    4
Name: 4.06 Developmental Disability, dtype: int64

In [301]:
6+15+33


54

In [0]:
df['is_minor'] = df['Age at Enrollment'] <= 17.0

In [0]:
df['under_6'] = df['Age at Enrollment'] < 6

In [304]:
df['under_6'].value_counts(normalize=False)

False    84
True     24
Name: under_6, dtype: int64

In [305]:
df['is_minor'].value_counts(normalize=False)

True     54
False    54
Name: is_minor, dtype: int64

In [306]:
df['3.5 Ethnicity'].value_counts()

Non-Hispanic/Latino    98
Hispanic/Latino        10
Name: 3.5 Ethnicity, dtype: int64

In [307]:
df['3.4 Race'].value_counts()

White                                        68
American Indian or Alaska Native             15
Multi-Racial                                 10
Black or African American                     9
Native Hawaiian or Other Pacific Islander     3
Client refused                                3
Name: 3.4 Race, dtype: int64

In [308]:
df['3.917 Living Situation'].value_counts(normalize=True)

Place not meant for habitation                                                                                          0.296296
Staying or living in a family member's room, apartment or house                                                         0.185185
Hotel or motel paid for without emergency shelter voucher                                                               0.166667
Staying or living in a friend's room, apartment or house                                                                0.129630
Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter    0.129630
Rental by client, no ongoing housing subsidy                                                                            0.037037
Client refused                                                                                                          0.037037
Substance abuse treatment facility or detox center                                               

In [309]:
df['3.917b Stayed Less Than 7 Nights'].value_counts()

Series([], Name: 3.917b Stayed Less Than 7 Nights, dtype: int64)

In [0]:
less_than_7 = df[df['Count of Bed Nights (Housing Check-ins)'] < 7]

In [311]:
less_than_7['3.12 Exit Destination'].value_counts()

No exit interview completed                                                                                             16
Emergency shelter, including hotel or motel paid for with emergency shelter voucher, or RHY-funded Host Home shelter     9
Client refused                                                                                                           7
Rental by client, no ongoing housing subsidy                                                                             7
Rental by client with RRH or equivalent subsidy                                                                          5
Name: 3.12 Exit Destination, dtype: int64

In [312]:
# 839 total

# minors percent = 51.25
# adults percent = 48.75

# exit to perm adults = 101
# exit to perm kids = 106
# exit to temp adults = 21
# exit to temp kids = 22
# exit to ES adults = 59 
# exut to ES kids = 62 

.1437 * 409

58.7733

Exit to Perm percent: 24.617737003058103 %
Unknown Exit percent: 45.87 %
Temp Exit percent: 5.05 %

Exits to ES percent: 14.37 %
Exits to Transitional 5.05 %

In [0]:
def served(X):
  X = X.copy()
  return(X['5.8 Personal ID'].value_counts().unique().sum())

In [0]:
def served_h(X):
  X = X.copy()
  return(X['5.9 Household ID'].value_counts().unique().sum())


In [315]:
served(df)

1

In [316]:
served_h(df)

21

In [317]:
need_to_exit.shape

(37, 149)

In [318]:
df['3.7 Veteran Status'].value_counts(normalize=False)

No                59
Client refused     1
Name: 3.7 Veteran Status, dtype: int64

In [319]:
df['R10 Pregnancy Status'].isnull().value_counts()

True    108
Name: R10 Pregnancy Status, dtype: int64